<a href="https://colab.research.google.com/github/V0000/AvitoParcers/blob/main/AvitoHousesParcer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

In [ ]:
#set global variable for chromedriver
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [ ]:
!pip install dateparser

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
from datetime import datetime
import dateparser
from tqdm import tqdm_notebook
from selenium import webdriver
#from selenium.webdriver.common.proxy import Proxy, ProxyType
import IPython

In [ ]:
#set maximum width in characters of a columns
pd.set_option('display.max_colwidth', -1)

In [1]:
root_url = 'https://www.avito.ru'
city = 'yablonovskiy' #krasnodar, krasnodarskiy_kray
type = 'doma_dachi_kottedzhi' #kvartiry, zemelnye_uchastki
main_url = root_url + '/' + city + '/' + type + '?proprofile=1&p='
page = 1
PROXY = '181.113.35.6:41116'
max_delay = 5 #delay between pages in seconds to simulate a real user

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('--proxy-server=%s' % PROXY)

driver = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


In [ ]:
#IP check
#driver.get('https://myip.ru/')
#IPython.display.HTML(data=driver.page_source)

In [ ]:
#main page display check
#driver.get(main_url + str(page))
#IPython.display.HTML(data=driver.page_source)

In [ ]:
#collect of all urls

all_urls = pd.DataFrame()
start_time = time.time()

while True:
  driver.get(main_url + str(page))
  time.sleep(random.uniform(0, max_delay))
  current_page = BeautifulSoup(driver.page_source, 'html.parser')
  links_on_page = current_page.findAll('a', attrs = {'data-marker':'item-title'})

  if len(links_on_page) == 0: 
    break

  for link in links_on_page:
    all_urls = all_urls.append({'url': link.get('href')}, ignore_index=True)
  
  print('current page: ',page, '    finded urls: ', len(links_on_page), '    total urls: ', len(all_urls), '    spended time: ', time.time() - start_time)
  page += 1

page = 1

In [ ]:
all_urls = all_urls.drop_duplicates()
#all_urls

In [ ]:
all_urls.to_csv(r'/content/drive/My Drive/Colab Notebooks/data/avitoHousesUrls.csv', index=False)  

In [ ]:
#all_urls = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/data/avitoHousesUrls.csv')
#all_urls

In [ ]:
houses_data = pd.DataFrame({'id':[],'title':[],'price':[],'seller':[],'date_of_public':[], 
                                         'views':[],'house_area':[],'walls_material':[],'land_area':[],'type':[],
                                         'distance_to_city':[],'floors':[],'adress':[],'url':[]})
#houses_data

In [ ]:
start_time = time.time()
for part_url in all_urls['url']:
  time.sleep(random.uniform(0, max_delay))
  driver.get(root_url + part_url)
  current_page = BeautifulSoup(driver.page_source, 'html.parser')

  try:
    id = int(current_page.find(attrs = {'data-marker':'item-view/item-id'}).get_text()[2:])
  except:
    id = 0
  try:
    title = current_page.find(attrs = {'class':'title-info-title-text'}).get_text()
  except:
    title = None
  try:
    price = float(current_page.find(attrs = {'class':'js-item-price'}).get_text().replace(" ", ""))
  except:
    price = 0
  try:
    seller = current_page.find(attrs = {'data-marker':'seller-info/label'}).get_text()
  except:
    seller = None
  try:
    date_of_public = dateparser.parse(current_page.find(attrs = {'class':'title-info-metadata-item-redesign'}).get_text().replace("\n", "").replace("в ", "").strip()) 
  except:
    date_of_public = None
  try:
    views = int(current_page.find(attrs = {'class':'title-info-metadata-views'}).get_text().split('(')[0].strip())
  except:
    views = 0
  try:
    house_area = float(current_page.findAll(attrs = {'class':'item-params-list-item'})[0].get_text().split('(')[0].replace("\xa0м²", "").replace("Площадь дома: ", "").strip())
  except:
    house_area = 0
  try:
    walls_material = current_page.findAll(attrs = {'class':'item-params-list-item'})[3].get_text().replace(" Материал стен: ", "").strip()
  except:
    walls_material = None
  try:
    land_area = float(current_page.findAll(attrs = {'class':'item-params-list-item'})[1].get_text().replace(" Площадь участка: ", "").replace("\xa0сот. ", "").replace("\xa0сот. ", ""))
  except:
    land_area = 0
  try:
    type = current_page.findAll(attrs = {'class':'item-params-list-item'})[4].get_text().replace(" Вид объекта: ", "").strip()
  except:
    type = None
  try:
    distance_to_city = current_page.findAll(attrs = {'class':'item-params-list-item'})[2].get_text().replace(" Расстояние до города: ", "").strip()
  except:
    distance_to_city = None
  try:
    floors = int(current_page.findAll(attrs = {'class':'item-params-list-item'})[5].get_text().replace(" Этажей в доме: ", "").strip())
  except :
    floors = 0
  try:
    adress = current_page.find(attrs = {'class':'item-address__string'}).get_text().replace("\n", "").strip()
  except:
    adress = None
  url = str(root_url + part_url)

  temp = {'id':id,'title':title,'price':price,'seller':seller,'date_of_public':date_of_public, \
                                          'views':views,'house_area':house_area,'walls_material':walls_material,'land_area':land_area,'type':type,\
                                          'distance_to_city':distance_to_city,'floors':floors,'adress':adress,'url':url}
  houses_data = houses_data.append(temp, ignore_index=True)
  print('count: ',len(houses_data), '       spended time: ', (time.time() - start_time)/60, '         title: ', title)
houses_data.to_csv(r'/content/drive/My Drive/Colab Notebooks/data/AllAvitoHouses.csv', index=False)

In [ ]:
#houses_data = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/data/AllAvitoHouses.csv')
#all_urls = pd.read_csv(r'/content/drive/My Drive/Colab Notebooks/data/avitoHousesUrls.csv')
#houses_data.info()

In [ ]:
houses_data['Price_for_meter_LAND'] = houses_data.price/houses_data.land_area

In [ ]:
houses_data.loc[(houses_data.price > 3000000) & (houses_data.price < 4500000)&(houses_data.seller != 'Агентство')].sort_values('Price_for_meter_LAND').head(20)